In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob
from PIL import Image, ImageFilter, ImageOps

In [2]:
images = [cv2.imread(file) for file in glob.glob("C:/Users/devda/Major/Data/all-mias/JPEG/Preprocessed/*.jpg")]

In [3]:
#Convert to Grayscale
for i in range(322):
    images[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)

In [4]:
#Thresholding
ret = [0] * 322
thresh = [0] * 322
for i in range(322):
    ret[i], thresh[i] = cv2.threshold(images[i], 10, 255, cv2.THRESH_BINARY)

In [5]:
#Identifying the orientation of mammogram
f = [0] * 322
for i in range(322):
    t = 1
    for j in range(int(len(images[i])/3)):
        if thresh[i][80, j] > 0:
            for c in range(j+1, j+21):
                t = 0 
                if thresh[i][80, c] < 1:
                    t = 1
                    break
        if t == 0:
            break
    if t == 1:
        f[i] = 'Right'
    elif t == 0:
        f[i] = 'Left'

In [6]:
#Error for flipping
count = 0
for i in range(322):
    if i%2==0 and f[i]!='Right':
        count = count + 1
        #print(i)
    if i%2==1 and f[i]!='Left':
        count = count + 1
        #print(i)
print(count)

1


In [7]:
#Flipping
flipped = [0] * 322
for i in range(322):
    flipped[i] = images[i].copy()
    if f[i] == 'Right':
        flipped[i] = cv2.flip(flipped[i], 1)

In [8]:
#Crop
cropped = [0] * 322
for i in range(322):
    for j in range(int(len(flipped[i])/2)):
        if flipped[i][100, j] > 0:
            break
    cropped[i] = flipped[i][0:len(flipped[i]), j:len(flipped[i])]

In [10]:
#Pectoral muscle removal
for k in range(322):
    my = [0] * 300
    T = cropped[k].max()
    for i in range(300):
        for j in range(5, cropped[k].shape[0]):
            if cropped[k][i, j] >= T/2:
                continue
            if cropped[k][i, j] < T/2:
                m = j
                break
        my[i] = m
    mx = [0] * 300
    for j in range(300):
        for i in range(5, cropped[k].shape[0]):
            if cropped[k][i, j] >= T/2:
                continue
            if cropped[k][i, j] < T/2:
                m = i
                break
        mx[j] = m
    z = max(my)
    for i in range(max(mx)):
        for j in range(z):
            if cropped[k][i, j] >= T/2 and j < z:
                cropped[k][i, j] = 0
            if cropped[k][i, j] < T/2:
                if cropped[k][i, j+1] < T/2:
                    if cropped[k][i, j+2] < T/2:
                        if cropped[k][i, j+3] < T/2:
                            z = j

In [11]:
#Padding
padded = [0] * 322
color = [0, 0, 0]
for i in range(322):
    l = 1024 - cropped[i].shape[1]
    padded[i] = cv2.copyMakeBorder(cropped[i], 0, 0, l, 0, cv2.BORDER_CONSTANT, value=color)

In [12]:
#Flipping back
final = [0] * 322
for i in range(322):
    final[i] = padded[i].copy()
    if f[i] == 'Right':
        final[i] = cv2.flip(final[i], 1)

In [13]:
#Saving files
os.mkdir('WOPectoralNewNew')
for i in range(322):
    cv2.imwrite("WOPectoralNewNew/sample%03i.jpg" %(i+1), final[i])

In [ ]:
#Pectoral muscle removal
for i in range(322):
    for j in range(5, cropped[i].shape[0]):
        if cropped[i][j, 5] >= 200:
            continue
        if cropped[i][j, 5] < 200:
            x = j
            break
    for j in range(5, cropped[i].shape[1]):
        if cropped[i][5, j] >= 150:
            continue
        if cropped[i][5, j] < 150:
            y = j
            break
    cv2.line(cropped[i], (y, 0), (0, x), (0, 0, 0), thickness=10, lineType=8)
    for h in range(x):
        for k in range(y):
            cropped[i][h, k] = 0
            if cropped[i][h, k+1] == 0:
                if cropped[i][h, k+2] == 0:
                    if cropped[i][h, k+3] == 0:
                        break